In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import datetime, sys
import numpy as np

%matplotlib inline

In [2]:
def parse(t):
    string_ = str(t)
    try:
        return datetime.date(int(string_[:4]), int(string_[4:6]), int(string_[6:]))
    except:
        return datetime.date(1900,1,1)

In [15]:
def readAllFiles():
    allFiles = glob.iglob("data/atp_rankings_" + "*.csv")
    ranks = pd.DataFrame()
    list_ = list()
    for filen in allFiles:
        df = pd.read_csv(filen,
                         index_col=None,
                         header=None,
                         parse_dates=[0],
                         date_parser=lambda t:parse(t),
                         names = ["ranking_date", "ranking", "player_id", "ranking_points"])
        list_.append(df)
    ranks = pd.concat(list_)
    return ranks

In [16]:
def readPlayers():
    return pd.read_csv("data/atp_players.csv",
                       index_col=None,
                       header=None,
                       parse_dates=[4],
                       names = ["player_id", "first_name", "last_name", "hand", "birth_date", "country_code"],
                       date_parser=lambda t:parse(t))

In [24]:
ranks = readAllFiles()
ranks = ranks[(ranks['ranking']<100)]
ranks['player_id'] = ranks['player_id'].apply(lambda row: int(row))
players = readPlayers()
players = ranks.merge(players,right_on="player_id",left_on="player_id")
players.head()
# plRanks["B"] = plRanks["ranking_date"] - plRanks["birth_date"]
# plRanks["B"] = plRanks["B"].astype(int) / (365*24*3600*1000000000.0)
# agg = plRanks[["ranking_date","B"]].groupby("ranking_date")
# data = agg.mean()
# data

/Users/claudiucreanga/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2787: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ranking_date,ranking,player_id,ranking_points,first_name,last_name,hand,birth_date,country_code
0,2000-01-10,1,101736,4135,Andre,Agassi,R,1970-04-29,USA
1,2000-01-17,1,101736,4135,Andre,Agassi,R,1970-04-29,USA
2,2000-01-31,1,101736,5045,Andre,Agassi,R,1970-04-29,USA
3,2000-02-07,1,101736,5045,Andre,Agassi,R,1970-04-29,USA
4,2000-02-14,1,101736,5045,Andre,Agassi,R,1970-04-29,USA


In [72]:
aggregate = players[["ranking_date", "country_code"]].groupby(["ranking_date","country_code"]).size().to_frame()
aggregate = aggregate[(aggregate[0] > 3)]
aggregate
#aggregate = aggregate.set_index["ranking_date"]

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>0</th>
    </tr>
    <tr>
      <th>ranking_date</th>
      <th>country_code</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="9" valign="top">1973-08-23</th>
      <th>AUS</th>
      <td>17</td>
    </tr>
    <tr>
      <th>CZE</th>
      <td>4</td>
    </tr>
    <tr>
      <th>ESP</th>
      <td>4</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>7</td>
    </tr>
    <tr>
      <th>GBR</th>
      <td>7</td>
    </tr>
    <tr>
      <th>GER</th>
      <td>4</td>
    </tr>
    <tr>
      <th>ITA</th>
      <td>4</td>
    </tr>
    <tr>
      <th>RSA</th>
      <td>4</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>24</td>
    </tr>
    <tr>
      <th rowspan="8" valign="top">1973-09-13</th>
      <th>AUS</th>
      <td>17</td>
    </tr>
    <tr>
      <th>CZE</th>
      <td>4</td>
    </tr>
    <tr>
      <th>ESP</th>
      <td>4</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>7</td>
    </tr>
    <tr>
      <th>GBR</th>
      <td>7</td>
    </tr>
    <tr>
      <th>ITA</th>
      <td>4</td>
    </tr>
    <tr>
      <th>RSA</th>
      <td>4</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>24</td>
    </tr>
    <tr>
      <th rowspan="7" valign="top">1973-09-26</th>
      <th>AUS</th>
      <td>18</td>
    </tr>
    <tr>
      <th>CZE</th>
      <td>4</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>7</td>
    </tr>
    <tr>
      <th>GBR</th>
      <td>5</td>
    </tr>
    <tr>
      <th>ITA</th>
      <td>4</td>
    </tr>
    <tr>
      <th>RSA</th>
      <td>5</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>25</td>
    </tr>
    <tr>
      <th rowspan="6" valign="top">1973-10-15</th>
      <th>AUS</th>
      <td>17</td>
    </tr>
    <tr>
      <th>CZE</th>
      <td>4</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>7</td>
    </tr>
    <tr>
      <th>GBR</th>
      <td>6</td>
    </tr>
    <tr>
      <th>ITA</th>
      <td>4</td>
    </tr>
    <tr>
      <th>RSA</th>
      <td>5</td>
    </tr>
    <tr>
      <th>...</th>
      <th>...</th>
      <td>...</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">2017-01-02</th>
      <th>GER</th>
      <td>6</td>
    </tr>
    <tr>
      <th>RUS</th>
      <td>6</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>7</td>
    </tr>
    <tr>
      <th rowspan="7" valign="top">2017-01-09</th>
      <th>ARG</th>
      <td>9</td>
    </tr>
    <tr>
      <th>AUS</th>
      <td>4</td>
    </tr>
    <tr>
      <th>ESP</th>
      <td>10</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>12</td>
    </tr>
    <tr>
      <th>GER</th>
      <td>6</td>
    </tr>
    <tr>
      <th>RUS</th>
      <td>6</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>7</td>
    </tr>
    <tr>
      <th rowspan="7" valign="top">2017-01-16</th>
      <th>ARG</th>
      <td>9</td>
    </tr>
    <tr>
      <th>AUS</th>
      <td>4</td>
    </tr>
    <tr>
      <th>ESP</th>
      <td>10</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>12</td>
    </tr>
    <tr>
      <th>GER</th>
      <td>6</td>
    </tr>
    <tr>
      <th>RUS</th>
      <td>6</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>7</td>
    </tr>
    <tr>
      <th rowspan="6" valign="top">2017-01-30</th>
      <th>ARG</th>
      <td>9</td>
    </tr>
    <tr>
      <th>ESP</th>
      <td>10</td>
    </tr>
    <tr>
      <th>FRA</th>
      <td>11</td>
    </tr>
    <tr>
      <th>GER</th>
      <td>6</td>
    </tr>
    <tr>
      <th>RUS</th>
      <td>6</td>
    </tr>
    <tr>
      <th>USA</th>
      <td>8</td>
    </tr>
    <tr>
      <th rowspan="7" valign="top">2017-02-06</th>
      <th>ARG</th>
      <td>8</td>
    </tr>
    <tr>

In [78]:
da = pd.DataFrame(aggregate.to_records())

In [83]:
da_indexed = da.set_index("ranking_date")

In [110]:
pivoted = pd.pivot_table(da, values='0', columns='country_code', index="ranking_date")
pivoted.plot(figsize=(20,10))

In [96]:
#fig, ax = plt.subplots(1,1); da.groupby("country_code").plot(x="ranking_date", y="0", ax=ax)